In [1]:
import numpy as np
import pandas as pd
import fpkit.similarity as fpkit

# FPKit (Fingerprint Kit) usage example for CATA (check-all-that-apply) datasets

If you use this code, please cite our work: 

The input table contains CATA sensory evaluation data for multiple assessors and multiple products (samples).

The first cell reads the input data and produces two multi-sheet excel files: one for the assessors, one for the samples. Each sheet contains pairwise similarity values for each sample pair/assessor pair according to the 44 metrics implemented in FPKit, for one specific assessor/sample, respectively.

In [2]:
# Read the excel file with the first two columns as indexes
indata=pd.read_excel('./CATA_example.xlsx', header=0, index_col=[0,1])
# Get rid of last column (overall liking)
indata=indata.iloc[:,:-1]

# Currently handles two-level indexes
for lev in range(0,len(indata.index.levels)):
    with pd.ExcelWriter('CATA_'+indata.index.get_level_values(lev).name+'_44metrics.xlsx') as writer:
        # 'Fix' one of the index levels (assessor, sample) to iterate over the other one
        for fixed in indata.index.get_level_values(lev).unique():
            # One value from the fixed level picked
            dfSlice=indata[np.in1d(indata.index.get_level_values(lev), [fixed])]
            df=pd.DataFrame()
            # Calculate the similarity metrics
            for metric in fpkit.metrics:
                dm=[]
                counter=0
                indices=[]
                # Iterate over object pairs at the non-fixed index level
                for i in dfSlice.index:
                    counter+=1
                    for j in dfSlice.index[counter:]:
                        # Collect index pairs from the non-fixed index level
                        indices.append((i[1-lev],j[1-lev]))
                        # Append similarity to flattened similarity matrix (upper triangle only)
                        dm.append(fpkit.sim(*fpkit.get_abcdp(list(dfSlice.loc[i]),list(dfSlice.loc[j])),metric=metric,scale=True))
                # Collect to dataframe
                df[metric]=pd.Series(dm, name=metric)
            # Reattach index pairs and write to excel sheet
            df.index=indices
            df.to_excel(writer,sheet_name=str(fixed))



The second cell aggregates the primary results by copying the individual sheet contents to one large sheet, appending the name of the assessor and the row-wise average similarity as the last two columns. It also creates a second sheet by pivoting the first one, keeping the average similarities only.

In [3]:
############## Aggregate results to a single table

with pd.ExcelWriter('CATA_Assessor_collected.xlsx') as writer:
    xl = pd.ExcelFile('./CATA_Assessor_44metrics.xlsx')
    sheets=xl.sheet_names

    # Read list of columns
    df=pd.read_excel('./CATA_Assessor_44metrics.xlsx', sheet_name=sheets[0])
    columns=list(df.columns)

    # Append a column name for collecting the assessor names
    columns.append('Assessor')

    # Initiate dataframe to collect results
    collectiondf=pd.DataFrame(columns=columns)

    # Iterate over all assessors
    for sheet in sheets:
        df=pd.read_excel('./CATA_Assessor_44metrics.xlsx', sheet_name=str(sheet))
        df['Assessor']=sheet
        collectiondf=collectiondf.append(df)

    # Calculate row-wise average similarities
    collectiondf['mean']=collectiondf.iloc[:,:-1].mean(axis=1)
    # Write to excel
    collectiondf.to_excel(writer, sheet_name='All data')
    
    # Collect average similarities into second sheet, by pivoting first sheet
    collectiondf2=collectiondf.pivot(columns='Assessor', values='mean')
    
    # Reindex column names numerically
    collectiondf2.columns=pd.to_numeric(collectiondf2.columns)
    collectiondf2 = collectiondf2.reindex(sorted(collectiondf2.columns), axis=1)
    # Reindex row names based on last input sheet
    collectiondf2 = collectiondf2.reindex(df.index, axis=0)
    
    # Write to excel
    collectiondf2.to_excel(writer, sheet_name='Mean similarities')